In [1]:
import pandas as pd
import scanpy as sc

In [2]:
# read MicroSPLIT dataset
microsplit_data_path = 'C2_non_log_transformed.h5ad'
adata = sc.read_h5ad(microsplit_data_path)
microsplit_df = adata.to_df() # cell-by-gene matrix

In [3]:
name_converter = {
    'ecpR': 'matA',
    'csiR': 'glaR',
    'yneJ': 'ptrR'
}
microsplit_df = microsplit_df.rename(columns=name_converter)
microsplit_df.index = microsplit_df.index.map(adata.obs['CRISPR_perturbation'].to_dict())
microsplit_df = microsplit_df.sort_index()

In [4]:
genes_not_in_microsplit = []
for tf_name in ['fadR', 'lacI', 'tyrR']:
    # read SERGIO dataset
    sergio_data_path = f'../{tf_name}_all_ancestors_eqns_and_params_3_bins_sergio_data_noise.csv'
    sim_data = pd.read_csv(sergio_data_path, index_col=1, ).drop(columns=['Unnamed: 0']).T
    # if a gene is in simulated data but not in MicroSPLIT data, set its value as 0 in MicroSPLIT data
    for gene_name in sim_data.columns:
        if gene_name not in(microsplit_df.columns):
            genes_not_in_microsplit.append(gene_name)
            microsplit_df[gene_name] = 0
    # extract Microsplit data that match the SERGIO output format
    output_df = microsplit_df.loc[[tf_name, 'MG1655'], sim_data.columns].T.reset_index()
    output_df.to_csv(f'{tf_name}_MicroSPLIT_all_ancestors_data.csv')
print('genes not in MicroSPLIT dataset:')
print(genes_not_in_microsplit)

genes not in MicroSPLIT dataset:
['c0293', 'ydeO']
